In [0]:
import re
import pandas as pd
import numpy
import json
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError, Request
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

# 언어 확인
def lang(url):
    html = get_html(url)
    bsObj = BeautifulSoup(html, "html.parser")
    for html in bsObj.findAll('html'):
        if html.get('lang') is None:
            return False
        elif 'en' in html.get('lang'):
            return True
        else:
            return False

# http 403 에러
def get_html(url):
    headers = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"}

    html = ""
    ret = requests.get(url, headers=headers)

    if ret.status_code == 200:
        html = ret.text

    return html



def link_text(bsObj):
    result = ""

    allText = bsObj.findAll("a")

    for link in allText:
        if 'href' in link.attrs:
            # print(link)
            result += " " + link.get_text()

    return result;



# title 텍스트
def title(bsObj):

    if bsObj.head is None:
        print("no header")
        raise Exception
        return ""
    elif bsObj.head.title is None:
        return ""
    else:
        return bsObj.head.title.get_text()


# description 텍스트
def description(bsObj):

    result = ""

    if bsObj.head is None:
        return result

    else:
        for meta in bsObj.head.find_all('meta'):
            if meta.get('name') == "description":
                result += " " + meta.get('content')

        return result


# 키워드
def keyword(bsObj):

    result = ""
    if bsObj.head is None:
        return result

    else:
        for meta in bsObj.head.find_all('meta'):
            if meta.get('name') == "keywords":
                result += " " + str(meta.get('content'))

        return result


# 모든 메타데이터
def meta_data(bsObj):

    result = ""
    metadatas = set()

    if bsObj.head is None:
        return result
    else:
        for meta in bsObj.head.find_all('meta'):
            # print(meta.get('content'))
            metadatas.add(meta.get('content'))

        for data in metadatas:
            result += " " + str(data)

        return result
@timeout(5)
def page_text(url):
    
    
    html = get_html(url)
    bsObj = BeautifulSoup(html, "html.parser")

    title2 = title(bsObj)
    link_text2 = link_text(bsObj)
    keyword2 = keyword(bsObj)
    description2 = description(bsObj)
    meta_data2 = meta_data(bsObj)

    return (title2 + link_text2 + keyword2 + description2)
#     return (title2 + link_text2 + meta_data2)
    
    
print(page_text("https://www.casinogates.co.uk/"))

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import nltk

nltk.download('stopwords')

# review = page_text("https://www.casinogates.co.uk/")

def preprocessing( review, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()	

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()

    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.	
        clean_review = ' '.join(words)
        
        print('num of words: ', len(clean_review.split()))
        
        if len(clean_review.split()) < 35:
          raise Exception
          
        

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review
  
# print(preprocessing(review, True))







[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [1]:
clean_train_reviews = []
# for review in train_data['review']:
clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

# 전처리한 데이터 출력
clean_train_reviews[0]


tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)
                                            
print(text_sequences[0])   


word_vocab = tokenizer.word_index
print("전체 단어 개수: ", len(word_vocab) + 1)

NameError: ignored

In [0]:
from functools import wraps
import errno
import os
import signal

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


In [0]:
import csv

DATA_PATH = '/content/gdrive/My Drive/projects/web classification/'




def write_csv(file_name, output_name, gamble):

    page_data = pd.read_csv(DATA_PATH + file_name, header = None, encoding='CP949')
    rows = page_data.shape[0]

    if gamble == 'gamble':
      category = 1
    elif gamble == 'normal':
      category = 0
    elif gamble == 'porn':
      category = 2
    else:
      print('wrong category')
      
      
      
    success = 0
    error = 0
    csvFile = open(DATA_PATH + output_name, 'a', newline='')

    try:
        writer = csv.writer(csvFile)


        for i in range(7500, rows):

            url = 'http://' + page_data[0][i]
            

            try:
              if lang(url) is True:
                writer.writerow([url, preprocessing((page_text(url)), remove_stopwords = True), category])
                success += 1
                print("page", i+1, " success")
              else:
                continue

            except requests.exceptions.ConnectionError:
                print("page", i+1,"Connection Error")

            except requests.exceptions.SSLError:
                print("page", i+1,"SSL Error")

            except Exception as e:
                print(e)
                error += 1
                print(url)
                # print('에러 인덱스 =', i)


    finally:
        csvFile.close()
    print('total = ', error + success)
    print('success =', success)
    print('error =', error)




In [0]:
write_csv('text_data_new.csv', 'gamble')

In [0]:
write_csv('url/도박2.csv', 'gam_pages.csv', 'gamble')

In [0]:
def tuning(file_name):
  page_data = pd.read_csv(DATA_PATH + file_name, header = None)
  rows = page_data.shape[0]
  
#   csvFile = open(DATA_PATH + "gam_pages.csv", 'w', newline='')
  print(page_data[1][28])
  print(page_data.dropna())
  print(page_data.dropna()[1][28])
  for i in range(rows):
    print(len(page_data[1][i].split()))
    
tuning('text_data_new.csv')

In [0]:
()